# Домашнее задание

In [362]:
from scipy.stats import chi2_contingency
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from gensim.summarization import keywords
import summa
import RAKE
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import requests
from unicodedata import normalize
from nltk import everygrams
from statistics import mean
import yake
from collections import Counter

## Создание корпуса

Для корпуса я взяла тексты новостей с хабра, в качестве ключевых слов были взяты теги.

In [30]:
lks = []
resp = requests.get('https://habr.com/ru/news/').text
soup = BeautifulSoup(resp, 'html.parser')
news = soup.find_all('a', {'class': 'tm-article-snippet__title-link'})
for nw in news:
    lks.append('https://habr.com' + nw.get('href'))
print('\n'.join(lks))

https://habr.com/ru/news/t/587290/
https://habr.com/ru/news/t/587284/
https://habr.com/ru/news/t/587280/
https://habr.com/ru/company/itsumma/news/t/587278/
https://habr.com/ru/news/t/587268/
https://habr.com/ru/news/t/587266/
https://habr.com/ru/news/t/587258/
https://habr.com/ru/news/t/587252/
https://habr.com/ru/company/vk/news/t/587250/
https://habr.com/ru/company/epam_systems/news/t/587244/
https://habr.com/ru/company/etmc_exponenta/news/t/587232/
https://habr.com/ru/news/t/587230/
https://habr.com/ru/news/t/587210/
https://habr.com/ru/news/t/587206/
https://habr.com/ru/news/t/587202/
https://habr.com/ru/news/t/587200/
https://habr.com/ru/company/superjob/news/t/587196/
https://habr.com/ru/news/t/587198/
https://habr.com/ru/news/t/587188/
https://habr.com/ru/news/t/587186/


In [2]:
#для сохранения использованных ссылок, чтобы не менялись тексты и подобранные к ним слова
lks = ['https://habr.com/ru/news/t/587290/',
 'https://habr.com/ru/news/t/587284/',
 'https://habr.com/ru/news/t/587280/',
 'https://habr.com/ru/company/itsumma/news/t/587278/',
 'https://habr.com/ru/news/t/587268/',
 'https://habr.com/ru/news/t/587266/',
 'https://habr.com/ru/news/t/587258/',
 'https://habr.com/ru/news/t/587252/',
 'https://habr.com/ru/company/vk/news/t/587250/',
 'https://habr.com/ru/company/epam_systems/news/t/587244/',
 'https://habr.com/ru/company/etmc_exponenta/news/t/587232/',
 'https://habr.com/ru/news/t/587230/',
 'https://habr.com/ru/news/t/587210/',
 'https://habr.com/ru/news/t/587206/',
 'https://habr.com/ru/news/t/587202/',
 'https://habr.com/ru/news/t/587200/',
 'https://habr.com/ru/company/superjob/news/t/587196/',
 'https://habr.com/ru/news/t/587198/',
 'https://habr.com/ru/news/t/587188/',
 'https://habr.com/ru/news/t/587186/']

In [381]:
texts = []
kws = []
for lnk in lks:
    text = []
    resp = requests.get(lnk).text
    soup = BeautifulSoup(resp, 'html.parser')
    article = soup.find('div', {'xmlns': 'http://www.w3.org/1999/xhtml'})
    for tag in article.strings:
        text.append(tag)
    kw = soup.find_all('a', {'class': 'tm-tags-list__link'})
    texts.append(normalize('NFKC', ' '.join(text)))
    kws.append([normalize('NFKC', k.text) for k in kw])

Объединяю свои ключевые слова с уже выделенными. В большинстве своем они схожи, но появились некоторые новые. Еще изначальные ключевые слова иногда включают в себя какие-то с одной стороны, специфичные для области слова, но которые сложно назвать значимыми (например, в тексте про соцсети один раз перечисляются названия разных сетей, которые заблокировали и непонятно, можно ли их считать ключевыми). Я решила не исключать их, и посмотреть на объединение моих слов с выделенными на хабре

In [382]:
mine_kws = []
mine_kws.append(['Tesla', 'NHTSA', 'автопилот', 'обновление', 'электромобиль', 'ошибка'])
mine_kws.append(['Meta AI', 'ReSkin', 'искусственный интеллект', 'кожа для роботов'])
mine_kws.append(['Apple', 'Apple Fitness+', 'Apple Watch', 'тренировки'])
mine_kws.append(['RP2040', 'микроконтроллер', 'Raspberry Pi', 'процессор', 'плата', 'Pico', 'Iono RP', 'чип'])
mine_kws.append(['Узкомназорат', 'соцсети', 'социальные сети', 'ограничения'])
mine_kws.append(['виртуальная «Тройка»', 'Тройка', 'Samsung Pay', 'фокус-группа', 'тестирование'])
mine_kws.append(['конференция', 'NVIDIA GTC 2021', 'NVIDIA', 'метавселенная', 'искусственный интеллект'])
mine_kws.append(['Telegram', 'обновление'])
mine_kws.append(['веб-разработка', 'VK Team', 'трансляция'])
mine_kws.append(['EPAM', 'ВТБ-Розничный бизнес', 'вебинар', 'Project Hiring Week'])

for i in range(len(mine_kws)):
    kws[i].extend(mine_kws[i])

Лемматизируем тексты

In [383]:
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(m.parse(t)[0].normal_form)
    return ' '.join(lemmas)

corpus = [normalize_text(text) for text in texts]
kws = [set([normalize_text(k) for k in kw]) for kw in kws]

Проверим, сколько токенов в текстах, чтобы уложиться в 3-5к знаков. В итоге возьмем 10 текстов

In [384]:
l = 0
for text in corpus[:10]:
    l += len(text.split())
print(l)

4142


In [385]:
corpus = corpus[:10]
kws = kws[:10]

##  Выделение ключевых слов

Методы: rake, text-rank и yake

In [386]:
#rake
stop = stopwords.words('russian')
rake = RAKE.Rake(stop)
rake_kw = [set([i[0] for i in rake.run(text, minFrequency=1, maxWords=3) if i[1] > 0]) for text in corpus]

In [387]:
#text-rank
trank_kw = [set([i[0] for i in summa.keywords.keywords(text, ratio=0.1, language='russian', additional_stopwords=stop, scores=True)]) for text in corpus]

In [388]:
yk = yake.KeywordExtractor(lan='ru', n=1)#, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize)
yake_kw = [set([i[0] for i in yk.extract_keywords(text)]) for text in corpus]

## Фильтрация по шаблонам

Для начала напишем функцию, которая размечает шаблоны для ключевых фраз. На выходе, помимо списка с фразами и шаблонами, также выдается все встретившиеся шаблоны

In [389]:
def get_tags(kwds):
    kws_pos = []
    voc = []
    for art in kwds:
        artpos = []
        for k in art:
            kpos = []
            for word in k.split(' '):
                ps = str(m.parse(word)[0].tag.POS)
                if ps == 'None':
                    ps = str(m.parse(word)[0].tag)
                kpos.append(ps)
            artpos.append((k, kpos))
            voc.append(' '.join(kpos))
        kws_pos.append(artpos)
    return kws_pos, Counter(voc)

kw_pos = get_tags(kws)
rake_pos = get_tags(rake_kw)
trank_pos = get_tags(trank_kw)
yake_pos = get_tags(yake_kw)

Шаблоны изначально размеченных ключевых фраз. Большая часть ключевых слов состоит из одного слова - существительного или английского слова. Учитывая, что все тексты связаны с технологиями, это неудивительно, там много названий фирм/продуктов на английском.  

In [390]:
kw_pos[1]

Counter({'LATN': 28,
         'NOUN': 38,
         'NOUN NOUN': 1,
         'LATN LATN': 11,
         'NOUN PREP NOUN': 1,
         'ADJF NOUN': 5,
         'LATN UNKN': 1,
         'LATN LATN UNKN': 1,
         'PNCT NOUN PNCT': 2,
         'ADJF PNCT INFN UNKN UNKN PNCT': 1,
         'LATN LATN NUMB,intg': 1,
         'LATN LATN LATN': 2,
         'ADJF UNKN NOUN': 1})

Функция, которая фильтрует ключевые фразы по шаблону

In [391]:
def filter_kw(kwd_ps, templ):
    fil_art = []
    for art in kwd_ps[0]:
        fil_k = []
        for k in art:
            if k[1] == templ:
                fil_k.append(k[0])
        fil_art.append(set(fil_k))
    return fil_art

## Подсчет метрик 

In [395]:
def calc_metrics(pred_kws, name, filt=''):
    if filt != '':
        pred_kws = filter_kw(pred_kws, filt)
        true_kws = filter_kw(kw_pos, filt)
    else:
        true_kws = kws
    precision = [len(kw & pred_kw)/len(pred_kw) if len(pred_kw) != 0 else 0 for kw, pred_kw in zip(true_kws, pred_kws) if len(kw) != 0]
    recall = [len(kw & pred_kw)/len(kw) for kw, pred_kw in zip(true_kws, pred_kws) if len(kw) != 0]
    fscore = [2*pr*rcl/(pr + rcl)  if pr + rcl != 0 else 0 for pr, rcl in zip(precision, recall)]
    if mean(recall) + mean(fscore) + mean(precision) != 0:
        print(name + ' ' + ' '.join(filt))
        if filt == '':
            print('\n'.join(['Ключевые слова: ' + ', '.join(true_k) + '\nНайдено: ' + ', '.join(true_k & pred_k) for true_k, pred_k in zip(true_kws, pred_kws)]))
        print('Precision: ' + str(mean(precision)) +'\nRecall: ' + str(mean(recall)) + '\nF-score: ' + str(mean(fscore)) + '\n')

Метрики без фильтров. Итоги.

1) Rake предсказывает слишком много ненужного и из-за этого очень снижается точность. Однако это помогает с полнотой, она у него лучшая среди всех моделей. Я пробовала отфильтровывать слова по их итоговому скору (брать, например, те, у кого скор > 4, но это улучшение точности незначительно и не оправдывает ухудшение полноты). В целом можно сказать, что это худшая из моделей.

2) TextRank дает лучшую точность и худшую полноту (из-за установленного ratio предсказывает меньше слов, чем другие модели, и поэтому получается лучшая точность, но худшая полнота). Их соотношение можно немного поменять, изменив ratio в модели, однако я остановилась на этом, т.к. он показался лучшим. Ф-скор примерно такой же, как и у yake. 

3) Yake. Пожалуй, лучшая из моделей, она не так сильно отстает от rake по полноте, а от textrank - по точности. Однако у нее есть небольшая особенность - она предсказывает только ключевые фразы, состоящие из одного слова. Если ставить максимальное количество слов в фразе выше, то модель в основном выдает фразы именно этой длины, но большинство ключевых фраз у нас состоит из одного слова, и поэтому результат получается очень плохой. Получается, этот, на первый взгляд, недостаток, в итоге становится ее преимуществом, потому что одна из основных проблем других моделей -  предсказание большого количесвта лишних фраз, длиной больше 1.

In [393]:
calc_metrics(rake_kw, 'Rake')
calc_metrics(trank_kw, 'TextRank')
calc_metrics(yake_kw, 'Yake')

Rake 
Ключевые слова: nhtsa, автопилот, автопилот автомобиль, электромобиль, tesla, обновление, full-self driving, ошибка
Найдено: обновление, nhtsa, автопилот
Ключевые слова: кожа для робот, искусственный интеллект, искусственный кожа, робот, meta ai, reskin
Найдено: meta ai, робот, искусственный интеллект
Ключевые слова: здоровье, apple watch, фитнес, ios, fitness +, apple fitness +, apple, тренировка, apple one
Найдено: apple watch, fitness +, тренировка, apple
Ключевые слова: arduino, микроконтроллер, pico, iono rp, pimoroni, чип, adafruit, rp2040, процессор, плат, raspberry pi, sparkfun
Найдено: pimoroni, arduino, sparkfun
Ключевые слова: tiktok, twitter, ограничение, узбекистан, блокировка, соцсеть, « вконтакте », замедление, skype, youtube, социальный сеть, « одноклассник », facebook, instagram, telegram, узкомназорат
Найдено: tiktok, « вконтакте », youtube, skype, « одноклассник », facebook, instagram, twitter
Ключевые слова: тестирование, мосметро, транспортный карта, тройка, 

Метрики с фильтрацией.

Добавление фильтрации явно немного улучшает результаты, можно заметить, что особенно хороши показатели для LATN по сравнению с другими, а это один из двух самых распространенных шаблонов. Самый распространенный шаблон - NOUN, дает примерно такие же результаты, как и модели без фильтрации, но они везде немного улучшены. Можно заметить, что Yake и TextRank очень плохо справляются с предсказанием фраз длиной больше 1, у них совпадений вообще практически нет (ниже выведены только те шаблоны, результат для которых больше 0, то есть есть хотя бы одно совпадение в какм-нибудь из текстов). Rake в этом плане несколько лучше, он предсказывает 7 из имеющихся 13 шаблонов, однако и он покрывает не все.

In [396]:
for meth in [(rake_pos, 'Rake'), (trank_pos, 'TextRank'), (yake_pos, 'Yake')]:
    for filt in kw_pos[1]:
        calc_metrics(*meth, filt.split(' '))

Rake LATN
Precision: 0.3214285714285714
Recall: 0.4523809523809524
F-score: 0.32702741702741706

Rake NOUN
Precision: 0.08625522138680033
Recall: 0.38095238095238093
F-score: 0.12159376571141277

Rake LATN LATN
Precision: 0.35714285714285715
Recall: 0.2619047619047619
F-score: 0.2857142857142857

Rake ADJF NOUN
Precision: 0.041666666666666664
Recall: 0.125
F-score: 0.0625

Rake LATN UNKN
Precision: 1.0
Recall: 1.0
F-score: 1.0

Rake PNCT NOUN PNCT
Precision: 1.0
Recall: 1.0
F-score: 1.0

Rake LATN LATN LATN
Precision: 0.5
Recall: 0.5
F-score: 0.5

TextRank LATN
Precision: 0.475
Recall: 0.4166666666666667
F-score: 0.41000000000000003

TextRank NOUN
Precision: 0.17924603174603174
Recall: 0.2702380952380952
F-score: 0.20598068598068597

TextRank LATN LATN
Precision: 0.14285714285714285
Recall: 0.07142857142857142
F-score: 0.09523809523809523

Yake LATN
Precision: 0.4369047619047619
Recall: 0.48095238095238096
F-score: 0.37243589743589745

Yake NOUN
Precision: 0.1714979464979465
Recall: 0.

Методы решения проблем:

1) Для Yake можно попробовать объединить результаты, которые он выдает для 1, 2 и 3 слов, но тогда нужно как-то решить, в каком соотношении их добавлять, потому что если добавить все, то получится ситуация, как с rake.

2) Для всех моделей: сохранить соотношение кол-ва фраз, состоящих из 1, 2 и 3 слов, поскольку между ними есть явная разница, и большое кол-во предсказанных длинных фраз сильно ухудшает качество.

3) Улучшить морф. разметку и придумать, что делать с синонимами/транслитерацией/сокращениями. Это особенно важно для текстов, в которых есть иностранные названия, потому что можно часто встретить в одном тексте, например, VK/вк/Вконтакте. Допустим, они встретились все по одному разу, не хочется учитывать их как отдельные ключевые слова, поэтому можно, например, заменять их все на один какой-то вариант в текстах, чтобы их значимость повышалась.